In [1]:
from pyspark import SparkContext, SparkConf
from operator import add

conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 



In [2]:
df_users = sc.textFile('./moviedata/users.csv')
df_zipcodes = sc.textFile('./moviedata/zipcodes.csv')
df_rating = sc.textFile('./moviedata/rating.csv')
df_movies = sc.textFile('./moviedata/movies.csv')

# df_movie = sqlContext.read.format('com.databricks.spark.csv').load('./moviedata/movies.csv')

In [3]:
zipcode_user = df_users.map(lambda x: x.split(",")).map(lambda x:  (str(x[4])[1:-1],str(x[0])))
zipcode_user.take(5)

[('94560', '780'),
 ('48825', '781'),
 ('77081', '783'),
 ('91040', '784'),
 ('23322', '785')]

In [4]:
zipcode_state = df_zipcodes.map(lambda x: x.split(",")).map(lambda x:  (str(x[0])[1:-1], str(x[3])[1:-1]))
zipcode_state.take(5)

[('2574', 'MA'),
 ('1886', 'MA'),
 ('1472', 'MA'),
 ('2671', 'MA'),
 ('2672', 'MA')]

In [5]:
zipcode_user_state = zipcode_user.join(zipcode_state)
zipcode_user_state.take(5)

[('10707', ('95', 'NY')),
 ('98101', ('6', 'WA')),
 ('11753', ('924', 'NY')),
 ('6811', ('403', 'CT')),
 ('37076', ('874', 'TN'))]

In [6]:
user_state = zipcode_user_state.map(lambda x: x[1])
user_state.take(5)

[('95', 'NY'), ('6', 'WA'), ('924', 'NY'), ('403', 'CT'), ('874', 'TN')]

In [7]:
user_movieid_rating = df_rating.map(lambda x: x.split(",")).map(lambda x: (str(x[0]), (str(x[1]),int(x[2]))))
user_movieid_rating.take(5)

[('253', ('97', 4)),
 ('284', ('269', 4)),
 ('106', ('526', 4)),
 ('121', ('180', 3)),
 ('62', ('86', 2))]

In [8]:
user_movieid_rating_state = user_movieid_rating.join(user_state)
user_movieid_rating_state.take(5)

[('214', (('117', 4), 'NY')),
 ('214', (('137', 4), 'NY')),
 ('214', (('213', 4), 'NY')),
 ('214', (('302', 4), 'NY')),
 ('214', (('693', 3), 'NY'))]

In [9]:
movieid_state_rating_count = user_movieid_rating_state.map(lambda x: ((x[1][0][0],x[1][1]),(x[1][0][1],1))).reduceByKey(lambda x, y: (x[0]+ y[0], x[1] + y[1]))
movieid_state_rating_count.take(5)

[(('712', 'MD'), (3, 1)),
 (('375', 'MA'), (2, 1)),
 (('1090', 'TX'), (7, 3)),
 (('316', 'AZ'), (5, 1)),
 (('471', 'NY'), (44, 12))]

In [10]:
movieid_state_avgrating = movieid_state_rating_count.map(lambda x: (x[0][0], (x[0][1], float(x[1][0])/x[1][1])))
movieid_state_avgrating.take(5)

[('712', ('MD', 3.0)),
 ('375', ('MA', 2.0)),
 ('1090', ('TX', 2.3333333333333335)),
 ('316', ('AZ', 5.0)),
 ('471', ('NY', 3.6666666666666665))]

In [34]:
def kar_le_dukhi(x):
    y = x.split('\"')
    return y[0].split(',')[0:1] + y[2].split(',')[2:]

In [38]:
df_movies_cleaned = df_movies.map(lambda x : kar_le_dukhi(x))

In [66]:
i = 2
movieid_genre = df_movies_cleaned.filter(lambda x: int(x[i]) == 1).map(lambda x: (str(x[0]), i))
movieid_genre.collect()

[('281', 2),
 ('295', 2),
 ('298', 2),
 ('300', 2),
 ('313', 2),
 ('314', 2),
 ('326', 2),
 ('328', 2),
 ('334', 2),
 ('339', 2),
 ('343', 2),
 ('349', 2),
 ('350', 2),
 ('353', 2),
 ('358', 2),
 ('362', 2),
 ('363', 2),
 ('373', 2),
 ('374', 2),
 ('380', 2),
 ('385', 2),
 ('388', 2),
 ('391', 2),
 ('397', 2),
 ('398', 2),
 ('399', 2),
 ('403', 2),
 ('405', 2),
 ('426', 2),
 ('431', 2),
 ('435', 2),
 ('449', 2),
 ('450', 2),
 ('452', 2),
 ('453', 2),
 ('456', 2),
 ('2', 2),
 ('4', 2),
 ('17', 2),
 ('21', 2),
 ('22', 2),
 ('24', 2),
 ('27', 2),
 ('28', 2),
 ('29', 2),
 ('33', 2),
 ('39', 2),
 ('50', 2),
 ('53', 2),
 ('54', 2),
 ('62', 2),
 ('68', 2),
 ('73', 2),
 ('74', 2),
 ('79', 2),
 ('80', 2),
 ('82', 2),
 ('92', 2),
 ('96', 2),
 ('101', 2),
 ('110', 2),
 ('117', 2),
 ('118', 2),
 ('121', 2),
 ('127', 2),
 ('128', 2),
 ('144', 2),
 ('145', 2),
 ('147', 2),
 ('148', 2),
 ('161', 2),
 ('164', 2),
 ('172', 2),
 ('173', 2),
 ('174', 2),
 ('176', 2),
 ('177', 2),
 ('181', 2),
 ('183', 2)

In [131]:
movieid_genre.join(movieid_state_avgrating).filter(lambda s: s[1][1][0]=='WY').collect()

#genre_stateWithMaxAvgRating = movieid_genre_state_avgrating.reduceByKey()

[('546', (2, ('WY', 1.0))),
 ('515', (2, ('WY', 3.0))),
 ('298', (2, ('WY', 2.0))),
 ('127', (2, ('WY', 4.0)))]

In [72]:
#New
#Pseudocode:
#for each genre:
#  get list of movies in the above genre
#  join list of movies with rating
#  joining this with users to get zipcodes
#  join this with zipcodes to get state
#  reduce on basis of states to get average rating
#  return the best average rating state(requirement of question)


In [85]:
df_users = sc.textFile('./moviedata/users.csv').map(lambda x: x.split(","))
df_zipcodes = sc.textFile('./moviedata/zipcodes.csv').map(lambda x: x.split(","))
df_rating = sc.textFile('./moviedata/rating.csv').map(lambda x: x.split(","))
df_movies = sc.textFile('./moviedata/movies.csv')

In [86]:
def clean_df_movies(x):
    y = x.split('\"')
    return y[0].split(',')[0:1] + y[2].split(',')[2:]

In [116]:
movieid_userid_rating = df_rating.map(lambda x: (x[1] , (x[0], float(x[2]))))

In [118]:
userid_zipcode = df_users.map(lambda x : (x[0], x[4]))

In [120]:
zipcode_state = df_zipcodes.map(lambda x: (x[0], x[3]))

In [92]:
movieid_userid_rating.take(5)

[(u'97', (u'253', 4.0)),
 (u'269', (u'284', 4.0)),
 (u'526', (u'106', 4.0)),
 (u'180', (u'121', 3.0)),
 (u'86', (u'62', 2.0))]

In [107]:
#Clean df_movies
df_movies_cleaned = df_movies.map(lambda x: clean_df_movies(x))

In [132]:
for genre_id in range(1, 20):
    #  get list of movies in the above genre
    movie_list = df_movies_cleaned.filter(lambda x : int(x[genre_id]) == 1).map(lambda x : (x[0], genre_id))

    #  join list of movies with rating
    userid_rating_list = movie_list.join(movieid_userid_rating).map(lambda x : (x[1][1][0], x[1][1][1]))

    #  joining this with users to get zipcodes
    zipcode_rating_list = userid_rating_list.join(userid_zipcode).map(lambda x : (x[1][1], x[1][0]))

    #  join this with zipcodes to get state
    state_rating_list = zipcode_rating_list.join(zipcode_state).map(lambda x : (x[1][1], (x[1][0], 1)))

    #  reduce on basis of states to get average rating
    state_avg_rating_tuple = state_rating_list.reduceByKey(lambda x, y : (x[0]+y[0], x[1]+y[1]))
    state_avg_rating = state_avg_rating_tuple.map(lambda x: (x[0], x[1][0]/x[1][1]))
    
    

In [133]:
from pyspark.statcounter import StatCounter
state_avg_rating.aggregate(StatCounter(), StatCounter.merge, StatCounter.mergeStats).maxValue

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 191.0 failed 1 times, most recent failure: Lost task 0.0 in stage 191.0 (TID 450, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/local/lib/python2.7/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/lib/python2.7/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python2.7/dist-packages/pyspark/rdd.py", line 933, in func
    acc = seqOp(acc, obj)
TypeError: unbound me

In [115]:
movie_user_id_rating_list.take(10)

[(u'217', 3.0),
 (u'207', 2.0),
 (u'60', 4.0),
 (u'268', 2.0),
 (u'102', 2.0),
 (u'87', 3.0),
 (u'279', 2.0),
 (u'222', 2.0),
 (u'94', 3.0),
 (u'267', 3.0)]